In [59]:
from dotenv import load_dotenv

load_dotenv(".env")

True

In [60]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.agents.output_parsers.tools import ToolsAgentOutputParser
from langchain.agents.format_scratchpad.tools import format_to_tool_messages
from langchain_anthropic import ChatAnthropic

In [61]:
from tools_basic import tools

llm_anthropic = ChatAnthropic(
    model="claude-3-opus-20240229",
    # max_tokens=,
    temperature=0.9,
    # anthropic_api_key=os.environ.get("ANTHROPIC_API_KEY", "not_provided"),
    streaming=True,
    verbose=True,
)
llm_gpt_4 = ChatOpenAI(
    temperature=0.9,
    model="gpt-4-turbo-preview",
    verbose=True,
    streaming=True,
)


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use the tavily_search_results_json tool for information.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [62]:
from langsmith import Client

client = Client()

In [63]:
agent_claude_3 = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_to_tool_messages(x["intermediate_steps"])
    )
    | prompt
    | llm_anthropic.bind_tools(tools=tools)
    | ToolsAgentOutputParser()
)
agent_gpt_4 = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_to_tool_messages(x["intermediate_steps"])
    )
    | prompt
    | llm_gpt_4.bind_tools(tools=tools)
    | ToolsAgentOutputParser()
)

# agent = agent_claude_3
agent = agent_claude_3.with_fallbacks([agent_gpt_4])

from langchain.agents import AgentExecutor

executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)
res = executor.invoke(
    {"input": "btc行情"},
)
print(res["output"])



> Entering new AgentExecutor chain...


/Users/zhangleping/github.com/holynull/chat-langchain/.venv/lib/python3.10/site-packages/langchain_anthropic/chat_models.py:503: UserWarning: stream: Tool use is not yet supported in streaming mode.
  warnings.warn(



Invoking: `CryptocurrencyLatestQuote` with `BTC`




> Entering new MyAPIChain chain...
https://pro-api.coinmarketcap.com/v2/cryptocurrency/quotes/latest?symbol=BTC
{"status":{"timestamp":"2024-05-11T03:10:50.496Z","error_code":0,"error_message":null,"elapsed":36,"credit_count":1,"notice":null},"data":{"BTC":[{"id":1,"name":"Bitcoin","symbol":"BTC","slug":"bitcoin","num_market_pairs":11041,"date_added":"2010-07-13T00:00:00.000Z","tags":[{"slug":"mineable","name":"Mineable","category":"OTHERS"},{"slug":"pow","name":"PoW","category":"ALGORITHM"},{"slug":"sha-256","name":"SHA-256","category":"ALGORITHM"},{"slug":"store-of-value","name":"Store Of Value","category":"CATEGORY"},{"slug":"state-channel","name":"State Channel","category":"CATEGORY"},{"slug":"coinbase-ventures-portfolio","name":"Coinbase Ventures Portfolio","category":"CATEGORY"},{"slug":"three-arrows-capital-portfolio","name":"Three Arrows Capital Portfolio","category":"CATEGORY"},{"slug":"polychain-capital-portfolio","name":"

/Users/zhangleping/github.com/holynull/chat-langchain/.venv/lib/python3.10/site-packages/langchain_anthropic/chat_models.py:503: UserWarning: stream: Tool use is not yet supported in streaming mode.
  warnings.warn(


Bitcoin (BTC) 最新行情如下：
- 价格：$60,829.40
- 24小时交易量：$27,315,735,066.39
- 市值：$1,198,134,769,876.06
- 排名：1
- 流通供给量：19,696,640 BTC
- 总供给量：19,696,640 BTC
- 1小时内价格变动：0.04%
- 24小时内价格变动：-3.23%
- 7天内价格变动：-3.14%
- 30天内价格变动：-14.23%
- 60天内价格变动：-15.66%
- 90天内价格变动：27.09%

> Finished chain.
Bitcoin (BTC) 最新行情如下：
- 价格：$60,829.40
- 24小时交易量：$27,315,735,066.39
- 市值：$1,198,134,769,876.06
- 排名：1
- 流通供给量：19,696,640 BTC
- 总供给量：19,696,640 BTC
- 1小时内价格变动：0.04%
- 24小时内价格变动：-3.23%
- 7天内价格变动：-3.14%
- 30天内价格变动：-14.23%
- 60天内价格变动：-15.66%
- 90天内价格变动：27.09%
